In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/nursery/nursery.data

--2019-04-27 12:34:04--  https://archive.ics.uci.edu/ml/machine-learning-databases/nursery/nursery.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1059371 (1.0M) [application/x-httpd-php]
Saving to: ‘nursery.data’

nursery.data        100%[===================>]   1.01M  5.75MB/s    in 0.2s    

2019-04-27 12:34:04 (5.75 MB/s) - ‘nursery.data’ saved [1059371/1059371]



In [0]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import OrdinalEncoder
import sys
np.set_printoptions(threshold=1000)
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('nursery.data',header=None)
enc = OrdinalEncoder()
df1 = enc.fit_transform(df)
print(enc.categories_)

[array(['great_pret', 'pretentious', 'usual'], dtype=object), array(['critical', 'improper', 'less_proper', 'proper', 'very_crit'],
      dtype=object), array(['complete', 'completed', 'foster', 'incomplete'], dtype=object), array(['1', '2', '3', 'more'], dtype=object), array(['convenient', 'critical', 'less_conv'], dtype=object), array(['convenient', 'inconv'], dtype=object), array(['nonprob', 'problematic', 'slightly_prob'], dtype=object), array(['not_recom', 'priority', 'recommended'], dtype=object), array(['not_recom', 'priority', 'recommend', 'spec_prior', 'very_recom'],
      dtype=object)]


# One Hot Encoding of categorical values
# Ordinal Encoding of the last column - for Y 

In [5]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore',sparse='False')
label = df.iloc[:,-1]
df2 = df.iloc[:,0:8]
df2 = enc.fit_transform(df2)
enc1 = OrdinalEncoder()
label1 = label.values.reshape(-1,1)
label1 = enc1.fit_transform(label1)
print(enc1.categories_)
# print(df2.shape)
# print(enc.get_feature_names())

[array(['not_recom', 'priority', 'recommend', 'spec_prior', 'very_recom'],
      dtype=object)]


In [6]:
X = df2
y = label1
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
print(type(X_train))
print(X_train.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(9072, 27)


In [0]:
print(X_train)

  (0, 26)	1.0
  (0, 22)	1.0
  (0, 20)	1.0
  (0, 17)	1.0
  (0, 12)	1.0
  (0, 11)	1.0
  (0, 4)	1.0
  (0, 0)	1.0
  (1, 26)	1.0
  (1, 21)	1.0
  (1, 20)	1.0
  (1, 18)	1.0
  (1, 15)	1.0
  (1, 8)	1.0
  (1, 5)	1.0
  (1, 2)	1.0
  (2, 24)	1.0
  (2, 22)	1.0
  (2, 20)	1.0
  (2, 18)	1.0
  (2, 15)	1.0
  (2, 10)	1.0
  (2, 4)	1.0
  (2, 2)	1.0
  (3, 24)	1.0
  :	:
  (9068, 0)	1.0
  (9069, 24)	1.0
  (9069, 22)	1.0
  (9069, 19)	1.0
  (9069, 18)	1.0
  (9069, 12)	1.0
  (9069, 11)	1.0
  (9069, 5)	1.0
  (9069, 2)	1.0
  (9070, 24)	1.0
  (9070, 23)	1.0
  (9070, 19)	1.0
  (9070, 16)	1.0
  (9070, 12)	1.0
  (9070, 10)	1.0
  (9070, 5)	1.0
  (9070, 1)	1.0
  (9071, 24)	1.0
  (9071, 21)	1.0
  (9071, 20)	1.0
  (9071, 18)	1.0
  (9071, 13)	1.0
  (9071, 8)	1.0
  (9071, 5)	1.0
  (9071, 0)	1.0


In [0]:
from scipy.sparse import csr_matrix
X_train = csr_matrix(X_train).toarray()
X_test = csr_matrix(X_test).toarray()
y_train = csr_matrix(y_train).toarray()
y_test = csr_matrix(y_test).toarray()

# Summary

In [0]:
def classSeparation(dataset):
	separated = {}
	for i in range(len(dataset)):
		vector = dataset[i]
		if (vector[-1] not in separated):
			separated[vector[-1]] = []
		separated[vector[-1]].append(vector)
	return separated

def mean(nos):
	return sum(nos)/float(len(nos))

def stdev(nos):
	avg = mean(nos)
	variance = sum([pow(x-avg,2) for x in nos])/float(len(nos)-1)
	return math.sqrt(variance)

def summarize(dataset):
	summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
	del summaries[-1]
	return summaries

def summarizeByClass(dataset):
	separated = classSeparation(dataset)
	summaries = {}
	for classValue, instances in separated.items():
		summaries[classValue] = summarize(instances)
	return summaries

def probCalc(x, mean, stdev):
#   print("x: "+str(x))
#   print("mean: "+str(mean))
#   print("stdev: "+str(stdev))
  if stdev==0:
    if x == mean:
      return 1.0
    else:
      return 0.0
  exponent = math.exp(-(math.pow(x-mean,2.0)/(2.0*math.pow(stdev,2.0))))
  return (1.0 / (math.sqrt(2.0*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
	probabilities = {}
	for classValue, classSummaries in summaries.items():
		probabilities[classValue] = 1
		for i in range(len(classSummaries)):
			mean, stdev = classSummaries[i]
			x = inputVector[i]
			probabilities[classValue] *= probCalc(x, mean, stdev)
	return probabilities
			
def predict(summaries, inputVector):
	probabilities = calculateClassProbabilities(summaries, inputVector)
	bestLabel, bestProb = None, -1
	for classValue, probability in probabilities.items():
		if bestLabel is None or probability > bestProb:
			bestProb = probability
			bestLabel = classValue
	return bestLabel

def getPredictions(summaries, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = predict(summaries, testSet[i])
		predictions.append(result)
	return predictions

def getAccuracy(testSet, predictions):
	correct = 0
	for i in range(len(testSet)):
		if testSet[i][-1] == predictions[i]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

In [0]:
trainingSet = np.c_[X_train,y_train]
testSet = np.c_[X_test,y_test]
# trainingSet = pd.DataFrame(trainingSet)
# testSet = pd.DataFrame(testSet)
summaries = summarizeByClass(trainingSet)
	# test model
predictions = getPredictions(summaries, testSet)
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: ' + str(accuracy))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


Accuracy: 87.62860082304526


In [0]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(testSet[:,-1],predictions))
p = predictions
t = testSet[:,-1]

[[1297    0    0    0    0]
 [   0 1021    0  214   81]
 [   0    0    0    0    1]
 [   0  173    0 1001    0]
 [   0   12    0    0   88]]


In [0]:
from sklearn.metrics import precision_score,recall_score
print("Macro Precision: "+ str(precision_score(t, p, average='macro')))
print("Micro Precision: "+ str(precision_score(t, p, average='micro')))
print("Weighted Precision: " + str(precision_score(t, p, average='weighted')))


print("Macro Recall: "+ str(recall_score(t, p, average='macro')))
print("Micro Recall: "+ str(recall_score(t, p, average='micro')))
print("Weighted Recall: " + str(recall_score(t, p, average='weighted')))
print("Precision: "+ str(precision_score(t, p, average=None)))
print("Recall: "+ str(recall_score(t, p, average=None)))

Macro Precision: 0.6376231406511379
Micro Precision: 0.8762860082304527
Weighted Precision: 0.8822304890801321
Macro Recall: 0.7016952822812405
Micro Recall: 0.8762860082304527
Weighted Recall: 0.8762860082304527
Precision: [1.         0.84660033 0.         0.82386831 0.51764706]
Recall: [1.         0.77583587 0.         0.85264055 0.88      ]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Using sklearn library

In [0]:
# Required Python Machine learning Packages
import pandas as pd
import numpy as np
# For preprocessing the data
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
# To split the dataset into train and test datasets
from sklearn.model_selection import train_test_split
# To model the Gaussian Navie Bayes classifier
from sklearn.naive_bayes import GaussianNB
# To calculate the accuracy score of the model
from sklearn.metrics import accuracy_score

clf = GaussianNB()

clf.fit(X_train, y_train.ravel())
target_pred = clf.predict(X_test)

In [0]:
print(accuracy_score(y_test,target_pred))
# print(confusion_matrix(y_test,target_pred))
print("Precision: "+ str(precision_score(y_test, target_pred, average=None)))
print("Recall: "+ str(recall_score(y_test, target_pred, average=None)))

0.8379629629629629
Precision: [1.         0.83252662 0.         0.82590759 0.28901734]
Recall: [1.         0.65349544 0.         0.85264055 1.        ]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
